Dado que no se tiene una API de OpenAI, ya que es de pago, se pega el output de cada celda de código importante

## Vector Database setup

Remove old Weaviate DB files

In [1]:
!rm -rf ~/.local/share/weaviate


### Step 1 - Download sample data

In [2]:
import requests
import json

# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data

# Parse the JSON and preview it
print(type(data), len(data))

def json_print(data):
    print(json.dumps(data, indent=2))

json_print(data[0])

<class 'list'> 10
{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


### Step 2 - Create an embedded instance of Weaviate vector database

In [6]:
# Se crean los embeddings con OpenAI
import weaviate, os
from weaviate import EmbeddedOptions
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

client = weaviate.Client(
    embedded_options=EmbeddedOptions(),
    additional_headers={
        "X-OpenAI-BaseURL": os.environ['OPENAI_API_BASE'],
        "X-OpenAI-Api-Key": openai.api_key  # Replace this with your actual key
    }
)
print(f"Client created? {client.is_ready()}")

embedded weaviate is already listening on port 8079
Client created? True


In [7]:
json_print(client.get_meta())

{
  "hostname": "http://127.0.0.1:8079",
  "modules": {
    "generative-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "Generative Search - OpenAI"
    },
    "qna-openai": {
      "documentationHref": "https://platform.openai.com/docs/api-reference/completions",
      "name": "OpenAI Question & Answering Module"
    },
    "ref2vec-centroid": {},
    "reranker-cohere": {
      "documentationHref": "https://txt.cohere.com/rerank/",
      "name": "Reranker - Cohere"
    },
    "text2vec-cohere": {
      "documentationHref": "https://docs.cohere.ai/embedding-wiki/",
      "name": "Cohere Module"
    },
    "text2vec-huggingface": {
      "documentationHref": "https://huggingface.co/docs/api-inference/detailed_parameters#feature-extraction-task",
      "name": "Hugging Face Module"
    },
    "text2vec-openai": {
      "documentationHref": "https://platform.openai.com/docs/guides/embeddings/what-are-embeddings",
      "nam

## Step 3 - Create Question collection

In [8]:
# resetting the schema. CAUTION: This will delete your collection 
if client.schema.exists("Question"):
    client.schema.delete_class("Question")
class_obj = {
    "class": "Question",
    "vectorizer": "text2vec-openai",  # Use OpenAI as the vectorizer
    "moduleConfig": {
        "text2vec-openai": {
            "model": "ada",
            "modelVersion": "002",
            "type": "text",
            "baseURL": os.environ["OPENAI_API_BASE"]
        }
    }
}

client.schema.create_class(class_obj)

## Step 4 - Load sample data and generate vector embeddings

In [9]:
# reminder for the data structure
json_print(data[0])

{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


In [10]:
with client.batch.configure(batch_size=5) as batch:
    for i, d in enumerate(data):  # Batch import data
        
        print(f"importing question: {i+1}")
        
        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }
        
        batch.add_data_object(
            data_object=properties,
            class_name="Question"
        )

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
{'error': [{'message': 'update vector: send POST request: Post "http://jupyter-api-proxy.internal.dlai/rev-proxy/for_weaviate/v1/embeddings": dial tcp: lookup jupyter-api-proxy.internal.dlai: no such host'}]}
{'error': [{'message': 'update vector: send POST request: Post "http://jupyter-api-proxy.internal.dlai/rev-proxy/for_weaviate/v1/embeddings": dial tcp: lookup jupyter-api-proxy.internal.dlai: no such host'}]}
{'error': [{'message': 'update vector: send POST request: Post "http://jupyter-api-proxy.internal.dlai/rev-proxy/for_weaviate/v1/embeddings": dial tcp: lookup jupyter-api-proxy.internal.dlai: no such host'}]}
{'error': [{'message': 'update vector: send POST request: Post "http://jupyter-api-proxy.internal.dlai/rev-proxy/for_weaviate/v1/embeddings": dial tcp: lookup jupyter-api-proxy.internal.dlai: no such host'}]}
{'error': [{'message': 'update vector: send POST reque

El output correcto debería ser:

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10

In [11]:
count = client.query.aggregate("Question").with_meta_count().do()
json_print(count)

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 0
          }
        }
      ]
    }
  }
}


El output correcto debería ser:

{
  "data": {
  
    "Aggregate": {
    
      "Question": [
      
        {
        
          "meta": {
          
            "count": 10
            
          }
          
        }
        
      ]
      
    }
    
  }
  
}

## Let's Extract the vector that represents each question!

In [ ]:
# write a query to extract the vector for a question
result = (client.query
          .get("Question", ["category", "question", "answer"])
          .with_additional("vector")
          .with_limit(1)
          .do())

json_print(result)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "vector": [
              0.015805326,
              -0.0068935337,
              -0.0014539,
              -0.02767898,
              0.0101830335,
              0.02009085,
              -0.019330727,
              -0.011133188,
              -0.019999111,
              -0.020706812,
              0.018623026,
              0.017299362,
              0.0007343219,
              -0.023131344,
              0.0053863926,
              0.01174915,
              0.016106755,
              0.005540383,
              0.013158,
              0.009003532,
              -0.0195011,
              0.0024785923,
              -0.012004709,
              0.0030306648,
              0.017613897,
              0.018989982,
              0.023511406,
              -0.02551656,
              -0.011775361,
              -0.013931229,
              -0.013302161,
              -0.0011172505,
              -0.008806949,
              -0.014625824,
              -0.041649524,
              -0.024101157,
              0.0138788065,
              -0.01884582,
              0.037324686,
              0.00082851824,
              0.0087807365,
              0.023773517,
              0.0025080799,
              -0.015372843,
              0.00686077,
              0.025175814,
              0.017876009,
              0.003548335,
              -0.004501766,
              0.0065724473,
              0.0015259807,
              0.003150253,
              -0.003505742,
              0.01961905,
              -0.007732291,
              -0.011991603,
              0.0033189876,
              0.003024112,
              0.02770519,
              -0.008204091,
              -0.0018544393,
              0.012718963,
              -0.026329106,
              0.007719185,
              -0.027967302,
              0.0085972585,
              0.007227726,
              0.0017856349,
              -0.000560673,
              0.012850019,
              0.025922833,
              0.0064151804,
              -0.013498745,
              -0.00048900186,
              -0.0022033753,
              -0.022974078,
              0.017627003,
              0.023458984,
              -0.007253937,
              -0.003623692,
              0.017876009,
              -0.025346186,
              -0.027416868,
              0.013813279,
              0.022738177,
              -0.0012769747,
              -0.0059597613,
              -0.0012532208,
              -0.00057254994,
              0.01383949,
              -0.021113085,
              -0.0136035895,
              0.00541588,
              0.0070180367,
              -0.020117061,
              -0.018688554,
              0.0015743074,
              0.015844643,
              -0.021099979,
              -0.024376374,
              -0.018636132,
              0.008164775,
              0.01680135,
              -0.014376818,
              0.0031649969,
              -0.0109759215,
              0.009377041,
              0.03604034,
              0.0143375015,
              -0.011703281,
              -0.0036728382,
              0.044375487,
              0.032161087,
              -0.05541038,
              -0.018596815,
              -0.033052266,
              0.018085698,
              0.009377041,
              -0.0048654457,
              -0.00232624,
              0.03357649,
              0.016735822,
              0.016604766,
              -0.01383949,
              0.0134201115,
              0.01271241,
              0.00805993,
              -0.021978054,
              -0.018793399,
              -0.00039111957,
              0.019487994,
              0.011270797,
              0.017351784,
              -0.010510673,
              -0.00028217948,
              0.015621848,
              -0.010169928,
              0.025922833,
              -0.011133188,
              -0.024756435,
              0.0027210456,
              0.030011773,
              -0.0063824165,
              -0.014927253,
              -0.012928653,
              0.037010152,
              0.027810035,
              0.023550723,
              0.00601546,
              0.017548367,
              -0.001901947,
              -0.011126636,
              0.0387663,
              -0.0035679934,
              -0.0016513028,
              -0.020733023,
              0.0034664252,
              0.023039605,
              -0.01271241,
              -0.014494768,
              -0.009593283,
              0.013118683,
              0.019029299,
              0.020785445,
              0.0053929454,
              0.038215864,
              0.0069394032,
              -0.009462227,
              0.02267265,
              -0.013013839,
              -0.018334704,
              0.016539238,
              0.012437193,
              0.038032386,
              0.0023327929,
              0.012515826,
              0.0037318133,
              -0.009383594,
              -0.0016619511,
              -0.019723894,
              -0.01623781,
              0.03559475,
              -0.0029946244,
              0.021860104,
              -0.03716742,
              -0.026119417,
              -0.007070459,
              -0.008898688,
              0.007751949,
              -0.007384993,
              0.034913257,
              0.015818432,
              0.0010115867,
              -0.018740976,
              -0.6353584,
              -0.024572957,
              0.027757613,
              0.020719917,
              0.022659544,
              -0.01940936,
              -0.005337246,
              0.0062218728,
              -0.02041849,
              0.0010910393,
              -0.01769253,
              0.010110953,
              0.004793365,
              0.0067100557,
              -0.01062207,
              -0.02509718,
              0.008911793,
              0.0066674626,
              -0.013079367,
              -0.021414513,
              -0.0232624,
              0.009331171,
              0.010156822,
              0.00742431,
              0.005540383,
              0.0070245895,
              0.01508452,
              0.0043248404,
              -0.008538283,
              0.028570158,
              -0.012365112,
              0.019907372,
              -0.021978054,
              -0.009639152,
              0.035935495,
              -0.021807682,
              -0.0022901997,
              0.026499478,
              0.003430385,
              0.032318354,
              0.010117506,
              -0.020169483,
              0.018780293,
              -0.035752017,
              0.022449855,
              0.0026653467,
              0.013551167,
              0.0059433794,
              0.008132011,
              -0.020733023,
              -0.007004931,
              0.020864079,
              -0.019081721,
              -0.0026325828,
              -0.008132011,
              -0.0091149295,
              0.040286545,
              -0.0016128052,
              0.0070508006,
              -0.009809525,
              -0.0022148427,
              0.008027166,
              -0.004986672,
              -0.010163375,
              -0.011978498,
              0.0015669356,
              0.0041610207,
              0.04201648,
              -0.0048818276,
              -0.0073718876,
              0.028543947,
              0.019933583,
              0.0050980696,
              -0.0020919777,
              0.013262845,
              0.0206806,
              0.008073036,
              -0.03627624,
              -0.01496657,
              0.020811656,
              0.008531731,
              0.00017344413,
              -0.022384327,
              0.01508452,
              0.01271241,
              -0.021624204,
              0.00019013326,
              0.0017266598,
              0.009619494,
              0.021086873,
              0.030273885,
              0.029199226,
              -0.003472978,
              -0.016984828,
              -0.007103223,
              0.015543215,
              -0.014927253,
              -0.012463404,
              0.0046721385,
              -0.0073784404,
              -0.015464582,
              -0.024428796,
              0.020864079,
              -0.032161087,
              0.03944779,
              0.017220728,
              -0.026696062,
              0.011336325,
              0.036852885,
              -0.011926075,
              -0.010897288,
              -0.0059466558,
              -0.01813812,
              0.0033091584,
              0.013905018,
              -0.025228236,
              0.0013941058,
              0.01917346,
              0.023865256,
              -0.019212777,
              0.03271152,
              0.015818432,
              0.015136942,
              -0.0005999897,
              0.0019445401,
              0.035411272,
              -0.014389924,
              -0.03339301,
              -0.012155423,
              0.014743775,
              -0.017915325,
              -0.0015194279,
              0.015910171,
              -0.01015027,
              0.011441169,
              0.021611098,
              0.031715497,
              0.00017549188,
              0.0075291544,
              0.008669339,
              -0.014704458,
              -0.017522156,
              0.0033026056,
              -0.041964058,
              -0.0164475,
              -0.03604034,
              -0.010065083,
              0.0013416834,
              -0.027285812,
              -0.01938315,
              0.0041085985,
              -0.022620227,
              -0.022751283,
              -0.0032256104,
              0.011663964,
              0.002491698,
              -0.017495945,
              -0.0059597613,
              0.0030847252,
              -0.0072146202,
              0.02293476,
              0.010471356,
              -0.039107043,
              0.004436238,
              0.0026080098,
              -0.024848174,
              0.0019543692,
              -0.005284824,
              -0.03449388,
              -0.051583555,
              0.0028095082,
              -0.021165507,
              0.005111175,
              0.024599168,
              -0.006945956,
              -0.016670294,
              -0.010471356,
              -0.02246296,
              -0.008197539,
              0.014704458,
              0.0061760033,
              0.035935495,
              -0.0060940934,
              -0.012404429,
              0.030300096,
              -0.004619716,
              0.013564273,
              0.04495213,
              -0.01769253,
              0.044847287,
              -0.0195011,
              0.009599836,
              -0.017233834,
              0.0024949743,
              0.008053377,
              -0.0073063592,
              0.02065439,
              0.035752017,
              0.021113085,
              0.021558676,
              0.0015366289,
              -0.0053306934,
              0.00019044042,
              0.005524001,
              0.019094827,
              -0.0043346696,
              -0.002491698,
              -0.020352961,
              0.025254447,
              0.013813279,
              0.01532042,
              -0.029802084,
              -0.007063906,
              0.0049899486,
              0.019632155,
              0.018242965,
              0.006444668,
              -0.006274295,
              0.01026822,
              0.0023458984,
              0.0032714799,
              -0.03808481,
              0.016630977,
              -0.010707256,
              -0.021886315,
              0.013131789,
              0.007280148,
              0.053444546,
              -0.03037873,
              -0.041649524,
              -0.010772785,
              -0.01111353,
              -0.0075029433,
              0.013773962,
              0.011028344,
              -0.0033173494,
              0.01724694,
              -0.014992781,
              0.019776316,
              -0.016303338,
              0.006588829,
              0.01395744,
              0.032344565,
              0.0040856637,
              0.025555877,
              0.009475333,
              0.009658811,
              0.0131448945,
              0.010051978,
              0.0110676605,
              0.034624934,
              0.006621593,
              0.0063725873,
              -0.017102778,
              -0.003248545,
              -0.013433217,
              -0.0006515929,
              0.015281104,
              0.017430417,
              0.036643196,
              0.0073784404,
              -0.0057205847,
              0.005582976,
              0.0038694218,
              0.032816365,
              -0.015713587,
              -0.018360915,
              -0.0034664252,
              0.016735822,
              0.015228681,
              -0.014416135,
              -0.005003054,
              0.008793842,
              -0.0146127185,
              0.026774695,
              0.015071414,
              -0.030431151,
              0.016198494,
              0.010235456,
              0.008249961,
              0.0046295454,
              -0.019802528,
              0.0053569046,
              0.01179502,
              -0.013105578,
              -0.020392278,
              -0.0006122762,
              0.01484862,
              -0.010261667,
              0.055253115,
              0.03905462,
              0.022410538,
              0.015975699,
              0.0006941861,
              0.012561696,
              -0.015870854,
              0.035489906,
              -0.0010951348,
              0.019147249,
              -0.014206446,
              -0.0012302861,
              -0.009855394,
              -0.008787289,
              0.0034991892,
              0.0514525,
              -0.002983157,
              -0.012299584,
              -0.048307158,
              0.029566184,
              0.0060023544,
              -0.017011039,
              0.006562618,
              0.003928397,
              0.0010017576,
              -0.013013839,
              0.009442569,
              0.02269886,
              0.025070969,
              0.018505076,
              -0.0028111464,
              -0.028806059,
              -0.033471644,
              -0.017731847,
              0.007358782,
              0.07339124,
              0.0018822886,
              0.010373064,
              0.015333526,
              0.005658333,
              0.0024032353,
              0.0027980409,
              -0.015530109,
              0.007837135,
              -0.0020723194,
              -0.0045214244,
              -0.008354805,
              0.009829183,
              -0.021086873,
              0.034441456,
              -0.014219551,
              -0.01299418,
              -0.014743775,
              0.03132233,
              0.0076143406,
              0.02053644,
              0.01905551,
              0.016657189,
              0.049224548,
              -0.004449344,
              -0.01066794,
              -0.004056176,
              0.0043674335,
              0.0076471046,
              -0.02041849,
              -0.018623026,
              -0.014376818,
              -0.01275828,
              0.022266377,
              -0.010589306,
              -0.0013588846,
              -0.016879983,
              0.0067559252,
              0.036328662,
              -0.034703568,
              0.021139296,
              0.032134876,
              -0.0007212163,
              -0.021283457,
              -0.0061039226,
              -0.006172727,
              0.011120083,
              0.014101601,
              -0.026001466,
              -0.012830361,
              0.04243586,
              -0.025765566,
              0.0014301462,
              -0.01134943,
              0.016093649,
              -0.0071163285,
              -8.493029e-05,
              8.9384135e-05,
              -0.008747973,
              -0.011519803,
              -0.010989027,
              -0.019422466,
              0.016408183,
              -0.019776316,
              -0.024966124,
              0.017417312,
              -0.0073391236,
              0.011447722,
              -0.029435128,
              0.0067559252,
              -0.0023327929,
              -0.035280216,
              -0.0067493725,
              0.00029385163,
              0.029356495,
              -0.0009861947,
              0.022882339,
              -0.0027194074,
              0.0091149295,
              0.028727425,
              0.012050578,
              0.031951398,
              -0.01656545,
              -0.025123391,
              -0.016198494,
              0.0025244618,
              0.013944334,
              0.00029200866,
              -0.007719185,
              -0.0070835645,
              -0.0050063306,
              -0.017404206,
              0.027049912,
              -0.009671916,
              0.0032321631,
              -0.033550277,
              -0.011781914,
              0.011945734,
              -0.00039071002,
              -0.02699749,
              0.030273885,
              -0.026748484,
              0.015477687,
              -0.0039578844,
              0.012692751,
              0.0070901173,
              0.011546014,
              -0.029671028,
              -0.03341922,
              -0.008243408,
              0.0114346165,
              -0.021532465,
              -0.0033615807,
              -0.010890735,
              0.010923499,
              0.014022968,
              -0.012240609,
              0.009508097,
              -0.0046295454,
              0.004219996,
              -0.017102778,
              -0.021296563,
              0.015582532,
              0.026001466,
              -0.01964526,
              0.037744064,
              -0.0033533897,
              -0.0032026756,
              -0.007162198,
              0.024048734,
              0.014887936,
              0.018754082,
              -0.011768809,
              -0.019356938,
              -0.028570158,
              -0.0035385059,
              -0.023603145,
              0.00845965,
              0.0015816793,
              -0.009436016,
              -0.013800173,
              0.011015238,
              -0.0139705455,
              -0.008276172,
              0.001528438,
              -0.028229414,
              -0.0098488415,
              0.035489906,
              0.0034664252,
              -0.009429463,
              -0.020968923,
              -0.0060777115,
              0.01736489,
              0.011034897,
              -0.016945511,
              -0.028937114,
              -0.014271974,
              0.0110676605,
              0.033497855,
              0.010051978,
              0.028072147,
              -0.0048064706,
              0.03695773,
              0.03339301,
              -0.009888158,
              0.018544393,
              -0.0129614165,
              0.014586507,
              -0.03517537,
              0.009855394,
              0.033445433,
              0.0048326817,
              0.0041577443,
              0.0070311422,
              0.0035909282,
              0.016381972,
              -0.008643128,
              -0.022803705,
              -0.028989537,
              -0.0016046142,
              -0.008197539,
              -0.0011770447,
              -0.009855394,
              -0.008328594,
              -0.035280216,
              -0.02079855,
              0.031846553,
              0.021951843,
              0.010412381,
              -0.012135765,
              0.010228903,
              -0.021742154,
              0.012974522,
              -0.0023049435,
              -0.014625824,
              -0.031846553,
              -0.013852595,
              -0.0012065322,
              0.010019214,
              -0.00517015,
              -0.019304516,
              -0.009993003,
              -0.0066740154,
              0.009920922,
              -0.021584887,
              0.0028815889,
              0.0012949948,
              0.0056714388,
              -0.021912526,
              -0.021427618,
              -0.016028121,
              -0.015451476,
              -0.057035472,
              -0.03517537,
              0.0067821364,
              0.03672183,
              -0.030562207,
              0.029382706,
              -0.00222631,
              -0.035254005,
              0.0061792796,
              0.0096653635,
              0.02836047,
              0.012653435,
              0.0034828072,
              0.02189942,
              -0.006729714,
              -0.037822697,
              -0.00036122248,
              0.0043084584,
              0.0055207247,
              -0.006041671,
              0.01952731,
              0.007666763,
              -0.03601413,
              0.040941823,
              0.0077781603,
              -0.013524956,
              -0.02065439,
              0.012620671,
              -0.031872764,
              0.020103956,
              -0.019003088,
              0.0015022268,
              0.016303338,
              0.010766232,
              0.0005041552,
              0.015333526,
              0.0022852852,
              -0.03769164,
              -0.004491937,
              0.0067493725,
              -0.019723894,
              -0.014586507,
              -0.02234501,
              -0.01985495,
              -0.012718963,
              -0.017889114,
              -0.02906817,
              0.013243186,
              0.0075619183,
              0.011146294,
              -0.01151325,
              -0.0052717184,
              0.008400675,
              -0.011500144,
              -0.013996757,
              -0.0031699114,
              -0.0012319243,
              0.030142829,
              -0.029802084,
              0.001338407,
              -0.00013136293,
              0.023655567,
              0.0107924435,
              -0.011788467,
              -0.0043019056,
              -0.006048224,
              -0.011002133,
              -0.0128696775,
              0.020995134,
              0.0018020169,
              -0.029566184,
              -0.016630977,
              -0.009291855,
              0.011559119,
              -0.019553522,
              0.0011147932,
              -0.020274328,
              -0.018740976,
              -0.015189365,
              0.02563451,
              -0.006769031,
              0.004223272,
              0.004246207,
              -0.022318799,
              -0.011172505,
              0.015281104,
              -0.03520158,
              0.0096653635,
              -0.017915325,
              0.0166834,
              -0.015477687,
              -0.00040524904,
              -0.0020592138,
              -0.025398608,
              0.009947133,
              7.5408265e-05,
              -0.022082899,
              -0.02359004,
              0.0043739863,
              0.029854506,
              -0.016604766,
              -0.009154246,
              0.02201737,
              0.047363557,
              0.016421288,
              -0.015307315,
              -0.028229414,
              0.03221351,
              0.013708434,
              -0.013656012,
              0.0095736245,
              -0.03132233,
              0.015150048,
              0.0030355793,
              0.010798996,
              0.026093205,
              -0.016185388,
              -0.023131344,
              -0.011500144,
              0.04246207,
              -0.013813279,
              -0.010287878,
              -0.014743775,
              -0.027731402,
              -0.01543837,
              -0.0025064417,
              -0.0234983,
              -0.043851264,
              0.012489615,
              0.033261955,
              0.0075619183,
              0.008649681,
              -0.02394389,
              -0.04859548,
              0.018989982,
              -0.01781048,
              -0.043169774,
              -0.013099025,
              -0.01849197,
              0.040286545,
              0.005209467,
              -0.011041449,
              -0.024992336,
              0.026263578,
              -0.039133254,
              -0.008381017,
              0.0038694218,
              0.012646882,
              0.023078922,
              0.006487261,
              0.015608743,
              0.012024367,
              -0.007725738,
              0.010477909,
              -0.043405674,
              -0.018570604,
              -0.0350181,
              -0.0050522,
              -0.021191718,
              -0.004819576,
              -0.033471644,
              -0.0067493725,
              0.052474733,
              -0.038871143,
              -0.01463893,
              -0.0016955341,
              -0.042619336,
              -0.019422466,
              -0.013387348,
              -8.0476435e-05,
              0.00066265074,
              -0.0011754065,
              0.0055928053,
              -0.010261667,
              -0.013996757,
              0.0143375015,
              -0.0015055032,
              0.0065560653,
              -0.023393456,
              0.010281325,
              -0.00019402397,
              0.032816365,
              0.005261889,
              0.0071687507,
              0.010359959,
              -0.007601235,
              0.026800906,
              -0.0059794197,
              -0.02521513,
              0.024664696,
              -0.017168306,
              -0.019278305,
              -0.013800173,
              0.00862347,
              -0.01275828,
              0.0032174194,
              -0.011781914,
              -0.024494324,
              0.025005441,
              -0.010078189,
              0.026617428,
              -0.035935495,
              0.0023606422,
              -0.0067166085,
              -0.014455452,
              -0.026892645,
              0.027652768,
              0.0060449475,
              -0.015975699,
              0.0029815189,
              -0.0019674748,
              0.024140473,
              -0.0017315744,
              -0.00021439906,
              -0.014835514,
              -0.037508164,
              -0.01127735,
              0.0052815476,
              -0.02178147,
              -0.021584887,
              -0.020942712,
              -0.0042101666,
              -0.013577378,
              -0.0039513316,
              0.17236456,
              -0.0062185964,
              0.015346631,
              0.022056688,
              -0.015949488,
              0.02723339,
              0.0256083,
              -0.0039087385,
              0.0068149003,
              0.00080599304,
              -0.001545639,
              0.026499478,
              -0.033733755,
              0.0066084876,
              -0.0050554764,
              0.0015726693,
              -0.01271241,
              0.01026822,
              -0.0045738467,
              0.027757613,
              -0.0015652974,
              -0.008420333,
              0.00869555,
              -0.0066412515,
              0.037010152,
              -0.0068476642,
              0.0005774645,
              -0.001104145,
              7.786556e-05,
              0.0014678247,
              -0.014494768,
              -0.02993314,
              0.0008137745,
              0.0162116,
              -0.016106755,
              -0.01327595,
              -0.045738466,
              0.02838668,
              0.019029299,
              0.019396255,
              0.009147693,
              -0.023039605,
              0.011224927,
              -0.02258091,
              -0.0017512328,
              -0.0047474955,
              -0.008066483,
              -0.01304005,
              -0.011290455,
              0.019016193,
              -0.00709667,
              0.017089672,
              0.0047376663,
              0.019959794,
              0.0052160197,
              0.0009362297,
              0.023105133,
              -0.026774695,
              -0.013708434,
              0.006880428,
              -0.0022607122,
              0.039840955,
              0.020392278,
              0.0062284255,
              -0.009121482,
              0.019042404,
              -0.014717564,
              0.016748928,
              0.01760079,
              0.012424087,
              0.0044034743,
              -0.025909727,
              -0.0027800207,
              0.021231035,
              -0.02290855,
              -0.03973611,
              0.0077650547,
              0.01703725,
              -0.00068886194,
              -0.0006470879,
              0.020484017,
              -0.013125236,
              -0.005009607,
              0.0074701793,
              -0.0324232,
              0.010359959,
              0.010484462,
              0.0044001974,
              -0.01038617,
              -0.009540861,
              -0.012089895,
              -0.007961638,
              0.013551167,
              -0.014101601,
              0.02441569,
              0.009685022,
              -0.006693674,
              0.0060875406,
              -0.023288611,
              -0.041859213,
              -0.0033583043,
              -0.022358116,
              0.014232657,
              0.0036859438,
              -0.028176991,
              0.0076208934,
              -0.0031551677,
              0.0013810003,
              0.0038563162,
              -0.018426443,
              -0.0098488415,
              -0.026368422,
              0.0074046515,
              -0.014389924,
              0.022921655,
              -0.0020313645,
              -0.0016398354,
              -0.017233834,
              0.008636575,
              -0.0044231326,
              -0.009914369,
              -0.03176792,
              0.0067493725,
              -0.010746573,
              -0.020379173,
              -0.014586507,
              -0.005746796,
              0.02065439,
              0.013524956,
              -0.020012217,
              0.016879983,
              0.008590706,
              -0.015844643,
              0.008000955,
              0.008171327,
              0.00091902865,
              -0.021375196,
              -0.026001466,
              -0.003980819,
              -0.0086889975,
              0.01860992,
              0.0045312536,
              0.02542482,
              -0.017640108,
              0.010759679,
              -0.0141540235,
              0.0345463,
              0.0051242807,
              -0.016945511,
              -0.015700482,
              -0.036643196,
              -0.0133939,
              0.0040987693,
              -0.007915769,
              0.019422466,
              -0.004219996,
              -0.024612274,
              -0.027836246,
              0.00437071,
              0.009337724,
              -0.033497855,
              0.010143717,
              0.020221906,
              -0.02948755,
              -0.013852595,
              0.028255625,
              -0.1647109,
              -0.0049375263,
              0.019802528,
              -0.0076798685,
              0.008132011,
              -0.013013839,
              0.00975055,
              -0.001378543,
              -0.022803705,
              0.0041249804,
              0.0070245895,
              -0.0119850505,
              -0.043431886,
              -0.014009862,
              0.0085972585,
              -0.016630977,
              0.0011254415,
              -0.011080766,
              0.013865701,
              0.0026030953,
              0.015910171,
              -0.025713144,
              0.02406184,
              -0.008996979,
              -0.015032098,
              0.0071818563,
              0.016879983,
              0.0075881295,
              0.019487994,
              -0.029566184,
              -0.010228903,
              0.0036335213,
              0.008125458,
              -0.009193563,
              -0.01280415,
              0.00063193456,
              -0.028281836,
              -0.008761078,
              -0.01766632,
              0.03037873,
              0.009796419,
              0.026132522,
              0.009259091,
              0.00043821774,
              0.022449855,
              0.051662188,
              0.0016570365,
              0.0031453385,
              0.014403029,
              -0.036459718,
              0.02166352,
              -0.0113428775,
              0.021833893,
              0.0014383371,
              0.011054555,
              0.00461644,
              0.0055862525,
              0.03695773,
              0.012738622,
              -0.020588862,
              -0.015805326,
              -0.016788244,
              0.04151847,
              -0.025018547,
              -0.027783824,
              -0.0069328505,
              -0.032318354,
              0.011808125,
              -0.021113085,
              0.0114346165,
              -0.0077978186,
              -0.041151512,
              0.0036367977,
              -0.0066412515,
              0.011926075,
              0.011788467,
              -0.016866878,
              -0.019710789,
              -0.0076864213,
              -0.021191718,
              -0.023629356,
              0.03905462,
              0.0046131634,
              0.0166834,
              0.009901264,
              0.0061268574,
              -0.0021001687,
              -0.007640552,
              0.019121038,
              -0.0027964027,
              0.021348985,
              -0.042566914,
              -0.015556321,
              -0.012096448,
              0.005003054,
              0.008813501,
              -0.016853772,
              0.026381528,
              -0.00030470468,
              0.0225547,
              0.005782836,
              -0.015045203,
              -0.01813812,
              0.03357649,
              0.028675003,
              0.011133188,
              0.011827784,
              0.020313645,
              0.034834623,
              0.0047409427,
              0.009036296,
              0.015490793,
              0.0032993292,
              0.038006175,
              -0.016132966,
              0.03176792,
              -0.0014063923,
              -0.03556854,
              0.028517736,
              0.015399054,
              0.002283647,
              0.014036073,
              -0.015136942,
              0.004675415,
              -0.00083957607,
              -0.00697872,
              -0.097295806,
              -0.0042265486,
              -0.00070892985,
              0.02124414,
              0.021388302,
              0.0392381,
              9.84966e-05,
              0.03973611,
              -0.013800173,
              0.039159466,
              -0.03197761,
              -0.0006290677,
              0.015333526,
              -0.013813279,
              0.028858481,
              0.0055108955,
              -0.01066794,
              -0.012188187,
              -0.0031682733,
              0.024730224,
              -0.0077388436,
              -0.02018259,
              0.006828006,
              -0.0015440008,
              -0.010595859,
              -0.008446544,
              -0.014652035,
              0.0018298662,
              0.0025817987,
              -0.010078189,
              0.008132011,
              -0.031715497,
              -0.023314822,
              -0.0051996377,
              0.0013924676,
              -0.03651214,
              -0.05488616,
              0.019343833,
              0.026918856,
              -0.009396699,
              0.007011484,
              0.015071414,
              -0.016159177,
              -0.023458984,
              -0.0072015147,
              -0.012620671,
              -0.0013007285,
              0.002250883,
              -0.0195011,
              -0.022725072,
              -0.0040430706,
              -0.03449388,
              -0.026918856,
              -0.0067428197,
              0.019723894,
              0.002218119,
              3.4875682e-06,
              0.015608743,
              0.010825207,
              0.024153579,
              0.011958839,
              0.002699749,
              0.016604766,
              0.016513027,
              0.00485234,
              0.016198494,
              0.0042167194,
              0.014403029,
              0.013931229,
              -0.004380539,
              0.008492414,
              0.004711455,
              -0.025319975,
              -0.0038432106,
              -0.042750392,
              0.040076856,
              -0.035122946,
              -0.01727315,
              0.01520247,
              -0.0019756658,
              -0.010812102,
              -0.016879983,
              -0.0055338303,
              -0.006670739,
              0.019016193,
              0.008335147,
              -0.011061108,
              -0.016893089,
              0.008806949,
              -0.009947133,
              -0.0025048035,
              -0.004144639,
              0.04246207,
              -0.027023701,
              -0.013066261,
              0.0067821364,
              -0.02041849,
              0.0010533608,
              0.010818655,
              0.0092066685,
              -0.014704458,
              -0.0040266886,
              -0.048307158,
              0.019802528,
              0.006913192,
              -0.0018429719,
              0.0018659066,
              0.02267265,
              0.011061108,
              -0.0056091873,
              -0.0044657257,
              0.0008788928,
              -0.02148004,
              0.00027357892,
              -0.0058156,
              -0.005524001,
              -0.020287434,
              -0.011578778,
              0.012587907,
              0.006795242,
              0.015779115,
              0.02100824,
              0.0139705455,
              0.021283457,
              0.013289056,
              0.029671028,
              -0.004727837,
              0.0020330027,
              -0.0052815476,
              0.0022148427,
              -0.014992781,
              0.020366067,
              0.01419334,
              -0.043825053,
              -0.030798107,
              0.009645705,
              -0.0025686931,
              -0.028308047,
              0.010871077,
              0.00033828773,
              0.0051308335,
              0.01712899,
              -0.008852818,
              -0.006886981,
              0.0070901173,
              -0.009829183,
              -0.004396921,
              0.034651145,
              0.002956946,
              0.00437071,
              0.02862258,
              0.001960922,
              0.02530687,
              0.008426886,
              -0.019042404,
              -0.018439548,
              -0.0347822,
              -0.02969724,
              0.031925187,
              0.017286256,
              0.029251648,
              -0.0022787324,
              0.020484017,
              0.007719185,
              0.000693367,
              -0.006470879,
              -0.027757613,
              -0.005750072,
              -0.015779115,
              -0.0048883804,
              0.023917679,
              -0.014940359,
              -0.012607565,
              -0.013433217,
              0.025359292,
              -0.00998645,
              -0.013013839,
              -0.005707479,
              -0.0058352584,
              0.0018757358,
              -0.0041741263,
              0.014927253,
              -0.013564273,
              -0.0057631778,
              -0.0020903395,
              0.0062448075,
              0.0076798685,
              -0.0017004487,
              -0.0206806,
              0.007391546,
              -0.0054289857,
              -0.0014326035,
              -0.023078922,
              0.02210911,
              -0.039107043,
              0.019579733,
              0.0142457625,
              0.009141141,
              0.0061366865,
              -0.00020416033,
              0.010746573,
              0.037586797,
              0.0098685,
              0.0066510807,
              -0.0037514716,
              0.0016144434,
              -0.016578555,
              -0.027076123,
              -0.0056943735,
              -0.035987917,
              -0.02549035,
              0.003522124,
              0.0067428197,
              -0.017627003,
              -0.010962816,
              0.016264021,
              0.0034828072,
              0.014796197,
              -0.025044758,
              -0.005176703,
              -0.03997201,
              0.020510228,
              -0.0078043714,
              -0.00048654457,
              -0.0003667514,
              -0.009003532,
              0.013092472,
              0.0023409838,
              0.014140918,
              0.0042986292,
              0.008498967,
              -0.0075029433,
              0.018649237,
              -0.0122602675,
              -0.024468113,
              -0.018203648,
              -0.01327595,
              0.005838535,
              0.023275506,
              0.0020624902,
              -0.023904573,
              0.07548813,
              0.0061465157,
              -0.00377113,
              0.034126922,
              -0.00036654662,
              0.030562207,
              0.023249295,
              0.024114262,
              -0.01022235,
              -0.0035876518,
              0.0037350897,
              -0.02619805,
              0.008118905,
              -0.004095493,
              -0.050928276,
              0.01623781,
              -0.005219296,
              -0.0011467381,
              -0.020575756,
              0.002642412,
              0.02859637,
              0.010798996,
              0.0052061905,
              -0.009770208,
              -0.011591883,
              -0.0069394032,
              0.010989027,
              0.011906417,
              -0.006919745,
              -0.016054332,
              -0.006139963,
              0.020693706,
              -0.016997933,
              -0.019252094,
              -0.010556542,
              -0.024088051,
              0.0070245895,
              -0.01022235,
              0.008361358,
              -0.0069656144,
              0.014442346,
              0.012895889,
              0.0041249804,
              -0.009475333,
              0.010477909,
              -0.0042986292,
              -0.01611986,
              0.007221173,
              -0.02791488
            ]
          },
          "answer": "Antelope",
          "category": "ANIMALS",
          "question": "Weighing around a ton, the eland is the largest species of this animal in Africa"
        }
      ]
    }
  }
}

## Query time
What is the distance between the `query`: `biology` and the returned objects?

In [12]:
response = (
    client.query
    .get("Question",["question","answer","category"])
    .with_near_text({"concepts": "biology"})
    .with_additional('distance')
    .with_limit(2)
    .do()
)

json_print(response)

{
  "data": {
    "Get": {
      "Question": null
    }
  },
  "errors": [
    {
      "locations": [
        {
          "column": 6,
          "line": 1
        }
      ],
      "message": "explorer: get class: vectorize params: vectorize params: vectorize params: vectorize keywords: remote client vectorize: send POST request: Post \"http://jupyter-api-proxy.internal.dlai/rev-proxy/for_weaviate/v1/embeddings\": dial tcp: lookup jupyter-api-proxy.internal.dlai: no such host",
      "path": [
        "Get",
        "Question"
      ]
    }
  ]
}


{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.1979338
          },
          "answer": "DNA",
          "category": "SCIENCE",
          "question": "In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance"
        },
        {
          "_additional": {
            "distance": 0.20262384
          },
          "answer": "species",
          "category": "SCIENCE",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        }
      ]
    }
  }
}

In [ ]:
response = (
    client.query
    .get("Question", ["question", "answer"])
    .with_near_text({"concepts": ["animals"]})
    .with_limit(10)
    .with_additional(["distance"])
    .do()
)

json_print(response)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.19060922
          },
          "answer": "Elephant",
          "question": "It's the only living mammal in the order Proboseidea"
        },
        {
          "_additional": {
            "distance": 0.19267958
          },
          "answer": "the nose or snout",
          "question": "The gavial looks very much like a crocodile except for this bodily feature"
        },
        {
          "_additional": {
            "distance": 0.20524907
          },
          "answer": "Antelope",
          "question": "Weighing around a ton, the eland is the largest species of this animal in Africa"
        },
        {
          "_additional": {
            "distance": 0.21580887
          },
          "answer": "species",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        },
        {
          "_additional": {
            "distance": 0.23756218
          },
          "answer": "the diamondback rattler",
          "question": "Heaviest of all poisonous snakes is this North American rattlesnake"
        },
        {
          "_additional": {
            "distance": 0.24850732
          },
          "answer": "DNA",
          "question": "In 1953 Watson & Crick built a model of the molecular structure of this, the gene-carrying substance"
        },
        {
          "_additional": {
            "distance": 0.25417864
          },
          "answer": "wire",
          "question": "A metal that is ductile can be pulled into this while cold & under pressure"
        },
        {
          "_additional": {
            "distance": 0.2574646
          },
          "answer": "Liver",
          "question": "This organ removes excess glucose from the blood & stores it as glycogen"
        },
        {
          "_additional": {
            "distance": 0.2601539
          },
          "answer": "Sound barrier",
          "question": "In 70-degree air, a plane traveling at about 1,130 feet per second breaks it"
        },
        {
          "_additional": {
            "distance": 0.2697832
          },
          "answer": "the atmosphere",
          "question": "Changes in the tropospheric layer of this are what gives us weather"
        }
      ]
    }
  }
}


## We can let the vector database know to remove results after a threshold distance!

In [ ]:
response = (
    client.query
    .get("Question", ["question", "answer"])
    .with_near_text({"concepts": ["animals"], "distance": 0.24})
    .with_limit(10)
    .with_additional(["distance"])
    .do()
)

json_print(response)

{
  "data": {
    "Get": {
      "Question": [
        {
          "_additional": {
            "distance": 0.19060922
          },
          "answer": "Elephant",
          "question": "It's the only living mammal in the order Proboseidea"
        },
        {
          "_additional": {
            "distance": 0.19267958
          },
          "answer": "the nose or snout",
          "question": "The gavial looks very much like a crocodile except for this bodily feature"
        },
        {
          "_additional": {
            "distance": 0.20524907
          },
          "answer": "Antelope",
          "question": "Weighing around a ton, the eland is the largest species of this animal in Africa"
        },
        {
          "_additional": {
            "distance": 0.21580887
          },
          "answer": "species",
          "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification"
        },
        {
          "_additional": {
            "distance": 0.23756218
          },
          "answer": "the diamondback rattler",
          "question": "Heaviest of all poisonous snakes is this North American rattlesnake"
        }
      ]
    }
  }
}

## Vector Databases support for CRUD operations

### Create

In [13]:
#Create an object
object_uuid = client.data_object.create(
    data_object={
        'question':"Leonardo da Vinci was born in this country.",
        'answer': "Italy",
        'category': "Culture"
    },
    class_name="Question"
 )

UnexpectedStatusCodeError: Creating object! Unexpected status code: 500, with response body: {'error': [{'message': 'update vector: send POST request: Post "http://jupyter-api-proxy.internal.dlai/rev-proxy/for_weaviate/v1/embeddings": dial tcp: lookup jupyter-api-proxy.internal.dlai: no such host'}]}.

In [ ]:
print(object_uuid)

d470073f-f781-43ac-8e4d-ea68bcb3dc76

### Read

In [ ]:
# Obtener la información del objeto creado
data_object = client.data_object.get_by_id(object_uuid, class_name="Question")
json_print(data_object)

{
  "class": "Question",
  "creationTimeUnix": 1708072945269,
  "id": "d470073f-f781-43ac-8e4d-ea68bcb3dc76",
  "lastUpdateTimeUnix": 1708072945269,
  "properties": {
    "answer": "Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vectorWeights": null
}

In [ ]:
# Obtener el vector del objeto creado
data_object = client.data_object.get_by_id(
    object_uuid,
    class_name='Question',
    with_vector=True
)

json_print(data_object)

{
  "class": "Question",
  "creationTimeUnix": 1708072945269,
  "id": "d470073f-f781-43ac-8e4d-ea68bcb3dc76",
  "lastUpdateTimeUnix": 1708072945269,
  "properties": {
    "answer": "Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vector": [
    0.023100348,
    -0.013096159,
    -0.003328341,
    -0.048296083,
    -0.004522966,
    0.012355109,
    -0.017082503,
    -0.004104528,
    -0.0044399174,
    -0.034471646,
    0.010176674,
    0.025617367,
    -0.0036413711,
    -0.007876861,
    0.010534423,
    -0.002036293,
    0.012706469,
    0.0052799885,
    0.03904572,
    -0.008362377,
    0.0020043512,
    0.021528808,
    0.011154095,
    -0.010726075,
    0.005241658,
    0.001534806,
    0.0166992,
    -0.032401815,
    0.014910457,
    -0.03178853,
    -0.0075574424,
    0.00057096063,
    -0.009301468,
    -0.013402802,
    -0.018654043,
    -0.015229875,
    -0.007353015,
    -0.028441027,
    0.007985463,
    0.015958149,
    0.014744359,
    0.0123742735,
    -0.008381542,
    -0.03544268,
    -0.011729049,
    -0.00045636922,
    0.009908363,
    0.008375154,
    0.018999014,
    0.026805604,
    0.02297258,
    0.04042561,
    -0.021937665,
    -0.0025649308,
    0.0032740398,
    -0.0048647444,
    -0.026754497,
    -0.0166992,
    -0.0026703388,
    -0.012521206,
    0.0030264906,
    0.021222167,
    -0.01470603,
    0.012847013,
    -0.028671008,
    -0.020353349,
    -0.010770793,
    0.01956119,
    0.0037116434,
    0.016660871,
    0.027112246,
    0.0077107633,
    -0.0037308084,
    0.008164338,
    0.03587709,
    0.0015324104,
    -0.013875541,
    -0.0015459857,
    -0.023330329,
    -0.0026687419,
    -0.004976541,
    -0.044284184,
    -0.005050007,
    -0.006219079,
    0.009071486,
    0.0045549083,
    -0.033117313,
    0.04139664,
    -0.005599407,
    -0.013926648,
    -0.017465806,
    -9.0884554e-05,
    0.004190771,
    -0.00072587864,
    -0.011939865,
    -0.020391678,
    -0.010176674,
    0.025144627,
    -0.008253775,
    0.0073594027,
    0.0026767273,
    0.01129464,
    -0.0030360732,
    -0.023189785,
    -0.0132239275,
    0.0060465927,
    -0.007027208,
    -0.0052863765,
    0.029744254,
    -0.009122593,
    -0.025106296,
    0.045740735,
    0.026371194,
    -0.049803738,
    -0.012010137,
    -0.010432209,
    0.021950442,
    0.009499507,
    0.009065098,
    -0.007582996,
    0.013747773,
    0.02744444,
    0.0070910915,
    -0.024378022,
    0.022078209,
    0.05366231,
    -0.016277568,
    -0.011978195,
    -0.031226356,
    0.003756362,
    0.001654588,
    -0.012827848,
    -0.008994826,
    0.009403681,
    -0.00041165063,
    -0.0140033085,
    -0.011262697,
    0.025936784,
    -0.024940198,
    -0.018398507,
    0.00964644,
    0.011115765,
    -0.011454348,
    0.010802735,
    -0.02117106,
    0.047427263,
    0.015574847,
    0.007691598,
    -0.008407095,
    0.010732463,
    0.00531193,
    -0.028364366,
    0.0072124703,
    0.011077435,
    0.008579582,
    0.010055295,
    -0.040246736,
    0.015868712,
    -0.009550614,
    -0.018705148,
    -0.026026221,
    0.003154258,
    0.0012736814,
    -0.008732903,
    0.019637851,
    0.02838992,
    0.024480237,
    0.009652828,
    0.027112246,
    0.0012225744,
    -0.009774207,
    0.02854324,
    -0.036643695,
    0.025067966,
    0.008688184,
    0.028645454,
    0.0032740398,
    -0.0051234732,
    -0.0054716393,
    0.0077171517,
    -0.019433424,
    0.027725529,
    0.03344951,
    0.036720354,
    -0.0018414478,
    -0.019816726,
    0.020302242,
    -0.00014094343,
    0.020161698,
    -0.016788637,
    0.0068355566,
    0.0036573422,
    0.016648093,
    -0.012904509,
    -0.6500806,
    -0.039301258,
    -0.030740838,
    0.00413647,
    0.014233289,
    0.013939424,
    0.011850428,
    0.008355989,
    -0.014220513,
    0.0064394777,
    -0.004522966,
    0.026396748,
    -0.011792932,
    -0.006033816,
    -0.021707684,
    -0.015843159,
    -0.0012297613,
    -0.02838992,
    -0.00032660546,
    -0.0028396307,
    -0.015868712,
    0.0089692725,
    0.021311603,
    0.028568793,
    0.0069761006,
    0.02172046,
    0.036720354,
    0.007052761,
    -0.0011658777,
    0.018027982,
    -0.045587413,
    0.0097486535,
    0.017197493,
    0.0135050155,
    0.031430785,
    -0.006618352,
    -0.008566805,
    0.047887225,
    0.0066630705,
    0.018922353,
    0.00092791085,
    -0.0061296416,
    0.0073146843,
    0.011965418,
    0.006084923,
    -0.005430115,
    0.008368766,
    0.0141438525,
    0.0048040547,
    -0.0066119637,
    0.008055735,
    0.010400267,
    -0.01129464,
    -0.0067780614,
    0.008298493,
    0.0057655047,
    0.008847893,
    -0.0050436184,
    0.033219527,
    0.0035806817,
    -0.017031396,
    0.0066055753,
    0.008534864,
    -0.012278448,
    0.010342772,
    -0.000247749,
    -0.03293844,
    0.011217979,
    0.013479462,
    -0.0053534545,
    0.007397733,
    0.015178769,
    0.0032325154,
    -0.024556896,
    0.005570659,
    0.019688958,
    0.018334623,
    -0.005465251,
    -0.008815952,
    -0.010540811,
    -0.004516578,
    -0.00299934,
    -0.014680476,
    0.0077427053,
    0.024122488,
    -0.0007737914,
    -0.0028683783,
    0.026498962,
    0.005743145,
    0.001809506,
    0.019037344,
    0.0073721795,
    0.004171606,
    -0.022525394,
    0.0075063356,
    0.035749324,
    0.0024675082,
    -0.009799761,
    -0.008566805,
    -0.020685544,
    -0.0070719263,
    0.013952201,
    -0.0021816285,
    -0.013147267,
    0.006995266,
    0.006995266,
    -0.014386611,
    0.018999014,
    0.014987118,
    -0.040093414,
    0.00088159513,
    0.015779275,
    -0.007065538,
    0.013006723,
    0.005276794,
    -0.02913097,
    0.0227426,
    -0.004340898,
    0.0095378375,
    -0.018168526,
    -0.021260498,
    -0.005452474,
    0.0026399943,
    -0.02317701,
    -0.0036573422,
    0.032350708,
    -0.006340458,
    -0.023036465,
    -0.032631796,
    0.037359193,
    -0.009914751,
    0.020263912,
    0.036362607,
    -0.023355883,
    0.021286052,
    0.005602601,
    -0.0018047147,
    -0.0096783815,
    -0.00027869266,
    0.005091531,
    -0.02556626,
    -0.014961564,
    0.005149027,
    -0.01089856,
    -0.010636637,
    -0.036873676,
    0.010004188,
    -0.00061048864,
    -0.053253457,
    0.016226461,
    -0.0157665,
    -0.023189785,
    -0.012738411,
    0.0047178115,
    0.012917286,
    0.0020586525,
    0.006088117,
    -0.0038362166,
    0.032018512,
    -0.016226461,
    -0.0029226795,
    0.014079968,
    -0.025234064,
    0.01580483,
    0.008739291,
    0.0013543345,
    -0.0051426385,
    -0.0037020608,
    0.009295079,
    -0.034420542,
    0.009595333,
    -0.016686425,
    0.023943612,
    0.001621049,
    -0.009588945,
    -0.0075766076,
    -0.018066311,
    -0.0104577625,
    0.014578261,
    -0.0123040015,
    0.0061136708,
    -0.023573088,
    -0.00941007,
    -0.019101229,
    0.020021154,
    -0.026090106,
    0.034216113,
    0.013620006,
    0.004379228,
    0.02344532,
    -0.028722115,
    -0.007014431,
    0.016750308,
    -0.020723874,
    0.005241658,
    0.01101994,
    0.01430995,
    0.011959029,
    -0.00041604263,
    0.00022339333,
    0.013108936,
    0.021068847,
    0.047861673,
    0.008138784,
    -0.009525061,
    -0.023547534,
    -0.020877196,
    -0.024608003,
    0.02943761,
    0.021618247,
    0.013453908,
    -0.022870367,
    -0.00882234,
    -0.020366125,
    0.0055419113,
    0.000418039,
    0.018986238,
    -0.023023687,
    0.003583876,
    0.0155620705,
    0.0023046047,
    -0.010553588,
    -0.018551828,
    -0.015191546,
    -0.039071273,
    0.009141758,
    -0.025553482,
    0.0020889973,
    -0.00795991,
    -0.049394883,
    -0.0026607565,
    0.013977755,
    0.002691101,
    0.012029302,
    0.008253775,
    0.01062386,
    -0.013000335,
    -0.024173595,
    0.011952641,
    -0.007531889,
    0.0005805432,
    0.011511844,
    0.0227426,
    -0.008930942,
    0.027751083,
    0.004615598,
    0.029846467,
    -0.006228661,
    -0.040527824,
    0.012636197,
    -0.016967513,
    0.0108793955,
    -0.011090212,
    0.023074795,
    0.018717926,
    -0.019855056,
    0.005535523,
    -0.009857256,
    0.021004962,
    0.037384745,
    0.0096081095,
    0.003050447,
    -0.0035359631,
    0.0027805383,
    -0.008879835,
    -0.022832036,
    0.00451019,
    -0.0053981734,
    -0.0024802848,
    0.011147707,
    -0.024646334,
    -0.033219527,
    0.01591982,
    -0.02281926,
    0.017453028,
    -0.034727182,
    -0.007033596,
    0.004478248,
    0.02168213,
    0.007953522,
    -0.036797013,
    -0.024352469,
    0.018564604,
    0.0077682585,
    0.00041165063,
    -0.026550068,
    -0.0073849563,
    0.024812432,
    -0.029514272,
    0.02410971,
    0.04387533,
    0.01764468,
    0.01078357,
    -0.0132239275,
    0.0019979628,
    -0.008451815,
    0.031584103,
    -0.039301258,
    0.01588149,
    0.0007901616,
    0.006465031,
    -0.007167752,
    -0.0029242767,
    -0.02520851,
    0.028798774,
    -0.00066558836,
    -0.0080749,
    -0.007870473,
    0.011039104,
    0.0028571987,
    -0.012176234,
    0.019458978,
    0.00709748,
    0.019343987,
    0.030102002,
    -0.0066311285,
    0.008847893,
    0.02759776,
    0.015434303,
    -0.006484196,
    -0.013051441,
    -0.039147936,
    -0.039812326,
    -0.0121570695,
    0.11100433,
    0.0068483334,
    -0.009601721,
    0.020174474,
    -0.0028827523,
    -0.021694906,
    -0.0048328023,
    -0.011818485,
    0.022844814,
    -0.0050627836,
    -0.003810663,
    -0.0146421455,
    0.0015316119,
    0.0065480797,
    0.0028971261,
    -0.0039863433,
    0.016763084,
    0.013913871,
    0.025655696,
    -0.0045549083,
    -0.02387973,
    0.0049989,
    0.018321848,
    0.0052991533,
    -0.010298054,
    -0.024556896,
    0.033321742,
    -0.009576168,
    0.016162578,
    -0.009850867,
    -0.021490479,
    0.010815511,
    -0.010496093,
    -0.0048647444,
    -0.012776741,
    0.036055963,
    0.04006786,
    0.03866242,
    0.025553482,
    -0.011364912,
    0.033883918,
    0.0040949457,
    -0.0031462726,
    -0.016673647,
    0.017120833,
    -0.015383196,
    -0.018206857,
    0.027112246,
    0.0023828621,
    -0.023764739,
    0.045510754,
    -0.022844814,
    -0.0135050155,
    -0.026217872,
    0.01830907,
    0.015191546,
    5.4800243e-05,
    0.0014805049,
    0.0039032944,
    0.016558656,
    -0.025655696,
    -0.022218753,
    0.0019452588,
    -0.025055189,
    0.0064778076,
    -0.02803217,
    -0.026958924,
    -0.0046986467,
    -0.024467459,
    -0.000258729,
    -0.0011562951,
    -0.013198374,
    -0.00063284795,
    0.014067192,
    0.031507444,
    -0.0025617366,
    -0.0004120499,
    -0.02670339,
    0.015958149,
    0.010719686,
    0.008375154,
    -0.026192319,
    -0.03544268,
    -0.018436838,
    0.006254215,
    0.02897765,
    0.005778281,
    -0.016060364,
    -0.0071102567,
    0.00015302144,
    0.017925767,
    0.02626898,
    -0.008292105,
    -0.004618792,
    -0.0080749,
    0.0010987998,
    0.008375154,
    -0.004382422,
    0.0075063356,
    -0.01011279,
    0.024876315,
    -0.027469995,
    -0.014961564,
    0.0010029742,
    -0.003740391,
    0.0119207,
    -0.007972687,
    -0.0037435852,
    -0.027112246,
    -0.004676287,
    0.011058269,
    -0.012450934,
    0.029003203,
    -0.0024387606,
    -0.010943279,
    0.0017088891,
    -0.03485495,
    0.0080493465,
    0.0019117199,
    -0.025975116,
    -0.004721006,
    -0.013006723,
    0.02591123,
    0.005238464,
    -0.009895586,
    0.033321742,
    0.0071485867,
    0.016686425,
    -0.015319313,
    -0.0038362166,
    -0.0003307978,
    0.030255323,
    -0.01654588,
    -0.030996375,
    -0.0009622483,
    -0.019701734,
    -0.010598307,
    -0.014054416,
    -0.012042078,
    -0.0025617366,
    0.005474834,
    -0.0063500404,
    0.009199253,
    -0.0118759805,
    0.003052044,
    -0.029079864,
    0.019420646,
    -0.007340238,
    -0.0004970951,
    -0.0039032944,
    -0.030280877,
    -0.025119074,
    -0.0012002151,
    -0.011051881,
    -0.020289466,
    -0.022257084,
    -0.023688078,
    -0.011888757,
    0.016839745,
    0.00079974416,
    0.014131076,
    0.031124141,
    0.011083823,
    -0.014948787,
    -0.010649414,
    0.015229875,
    0.0015851145,
    -0.002708669,
    -0.015114885,
    0.020340571,
    0.037026998,
    0.020174474,
    -0.015025448,
    -0.025553482,
    -0.0076277144,
    0.011741825,
    -0.005880495,
    -0.013862764,
    -0.018794587,
    -0.018283516,
    -0.024148041,
    0.0023285612,
    -0.013326141,
    -0.004462277,
    0.0071485867,
    -0.00019953676,
    0.049343776,
    -0.0066247405,
    0.0003399811,
    -0.0006763687,
    0.0017360398,
    -0.017542467,
    0.03830467,
    -0.0031350928,
    0.007512724,
    -0.016941959,
    -0.020289466,
    0.0019228995,
    -0.0058389707,
    0.0076277144,
    0.005845359,
    0.017734116,
    -0.007027208,
    -0.0014102329,
    -0.028568793,
    0.010106402,
    0.0016657676,
    0.008771233,
    -0.011959029,
    -0.023253668,
    -0.013607229,
    0.0022327355,
    -0.00046076122,
    -0.013607229,
    -0.010674967,
    0.029463165,
    -0.011102988,
    0.010131956,
    -0.02780219,
    -0.0128661785,
    0.00795991,
    0.023125902,
    0.03493161,
    -0.007921579,
    0.004912657,
    0.00080373685,
    -0.0032021708,
    0.006065758,
    0.002001157,
    0.011454348,
    -0.015306536,
    0.010815511,
    0.007595773,
    0.004155635,
    -0.029948682,
    0.009269525,
    0.0016226461,
    -0.012610643,
    -0.02215487,
    0.028159937,
    0.0053758137,
    -0.0014749151,
    -0.022065433,
    -0.03953124,
    -0.0055131637,
    -0.011320192,
    -0.024416352,
    -0.0054109497,
    -0.015191546,
    -0.014067192,
    -0.008413484,
    0.018679596,
    -0.030485304,
    0.003328341,
    -0.009499507,
    -0.019062897,
    0.0025026442,
    -0.019037344,
    -0.004107722,
    0.017708564,
    -0.0029514271,
    0.0039607896,
    0.0011986181,
    0.007033596,
    0.03603041,
    0.0048168316,
    -0.01580483,
    0.023470873,
    -0.0019069286,
    0.024889091,
    -0.01658421,
    -0.024326915,
    -0.0067461194,
    -0.0044558886,
    0.017772447,
    0.0037499736,
    -0.00558663,
    -0.008343212,
    -0.01489768,
    -0.0147826895,
    0.011007163,
    0.0026096494,
    -0.003826634,
    0.01482102,
    -0.007423287,
    0.001034916,
    -0.0034241667,
    -0.00052105146,
    -0.012993946,
    0.009933916,
    -0.017427474,
    -0.0018462391,
    0.021809896,
    -0.025783464,
    -0.0010484913,
    -0.019714512,
    -0.019944493,
    0.026831156,
    -0.007499947,
    0.011582116,
    -0.00092471665,
    0.009799761,
    -0.0096783815,
    0.007927968,
    -0.009493119,
    -0.018436838,
    0.04356869,
    -0.0044015874,
    0.011882369,
    -0.021298828,
    0.017925767,
    0.024556896,
    0.018884024,
    -0.002860393,
    0.009729489,
    0.0074424515,
    -0.0036254004,
    -0.012680916,
    -0.018666819,
    0.028696561,
    -0.007602161,
    0.0062350496,
    -0.03045975,
    0.00041125136,
    0.010406655,
    -0.00039108808,
    0.0041779946,
    -0.002018725,
    -0.019126782,
    -0.028313259,
    -0.017631903,
    0.0009119399,
    -0.026754497,
    0.00983809,
    -0.016609764,
    0.002464314,
    -0.025349054,
    0.018922353,
    -0.0009862047,
    -0.01034916,
    0.0247741,
    0.027163351,
    0.012495653,
    0.02090275,
    0.020570554,
    -0.018845694,
    0.007033596,
    -0.021899335,
    -0.034624968,
    0.0037978864,
    -0.004324927,
    0.014539932,
    0.0011467126,
    0.008483756,
    -0.0275211,
    0.001388672,
    -0.05672873,
    -0.018500721,
    -0.015894266,
    0.03183964,
    0.013556123,
    0.012418993,
    -0.005462057,
    -0.0038074688,
    -0.0012273657,
    0.025451269,
    -0.015025448,
    -0.0073466264,
    0.01372222,
    -0.010125567,
    -0.0117354365,
    0.0055834358,
    -0.015510964,
    -0.024786878,
    0.009231196,
    0.009269525,
    -0.001541993,
    -0.0067141773,
    -0.032401815,
    -0.011000774,
    -0.011160484,
    -0.0013311767,
    0.005545106,
    0.024441905,
    0.013517792,
    -0.038049135,
    -0.0007929565,
    -0.033577275,
    0.0046092095,
    -0.034088347,
    -0.014335504,
    -0.009774207,
    0.009103429,
    0.0071038683,
    0.02031502,
    0.011933477,
    0.02258928,
    0.013530569,
    0.025515152,
    0.0008416678,
    -0.013390025,
    0.033960577,
    -0.011697107,
    0.006771673,
    -0.022244306,
    0.017823555,
    0.009442012,
    -0.015702615,
    -0.009588945,
    -0.014041639,
    0.024173595,
    -0.017350815,
    0.0062350496,
    -0.026524514,
    0.020557776,
    0.0049733464,
    0.01690363,
    -0.0019676182,
    0.022423182,
    0.001267293,
    -0.019050121,
    0.020021154,
    -0.009320633,
    -0.00017617928,
    -0.020008376,
    -0.016022034,
    -0.011154095,
    -0.009148147,
    0.0013758953,
    0.023675302,
    -0.011690718,
    -0.006912217,
    0.0015372017,
    -0.0060018743,
    0.00047393725,
    -0.0033538945,
    0.20095259,
    0.019062897,
    0.008867058,
    0.01603481,
    -0.027418887,
    0.033347294,
    0.031047482,
    0.01690363,
    0.005206522,
    -0.0026096494,
    -0.026077328,
    0.0060465927,
    -0.003032879,
    0.00032880146,
    0.012693692,
    -0.024365246,
    -0.021771567,
    -0.018922353,
    -0.0044239466,
    -0.00036513532,
    -0.0035998467,
    -0.009691158,
    0.006982489,
    -0.011971806,
    0.0076277144,
    0.020979408,
    -0.028875435,
    -0.012329555,
    0.02336866,
    -0.0035040213,
    0.01760635,
    -0.032989547,
    -0.020545,
    0.0038873234,
    -0.017900214,
    -0.0025441686,
    0.024314139,
    -0.019126782,
    0.010483316,
    -0.010464151,
    0.010777181,
    0.0076468796,
    -0.0042865966,
    -0.024173595,
    -0.017746894,
    0.02693337,
    0.0017009036,
    0.026320087,
    -0.015510964,
    0.011767379,
    -0.018641265,
    -0.030280877,
    0.017325262,
    0.035979304,
    0.018257963,
    -0.00095266575,
    0.020877196,
    0.0247741,
    0.0015587624,
    -0.0068163914,
    0.0035263805,
    0.02081331,
    -0.0020027542,
    -0.012847013,
    -0.014361057,
    -0.007991851,
    0.0015843159,
    0.010042518,
    0.008081289,
    -0.020493893,
    0.032325156,
    -0.03955679,
    -0.013453908,
    -0.000705915,
    -0.0071485867,
    -0.010802735,
    0.014246066,
    0.0068163914,
    -0.019535637,
    0.00031961818,
    0.019101229,
    0.010432209,
    -0.017453028,
    -0.0034401375,
    -0.022423182,
    -0.035825983,
    -0.0061871368,
    -0.0054588625,
    -0.0066502937,
    -0.023573088,
    -0.01379888,
    -0.025834572,
    -0.0036254004,
    0.010681355,
    0.008464591,
    0.0021081623,
    -0.0068163914,
    0.005500387,
    -0.01885847,
    -0.020672766,
    -0.025655696,
    -0.024263032,
    0.011850428,
    -0.0031414812,
    -0.012482876,
    0.012361497,
    -0.019944493,
    0.024007497,
    0.032682903,
    -0.015012671,
    0.011780156,
    0.006880275,
    3.426263e-05,
    -0.0062925452,
    0.010138344,
    0.007123033,
    -0.00039248553,
    -0.020596107,
    0.0132239275,
    -0.011748213,
    0.03155855,
    -0.00051825657,
    0.0065608565,
    0.020557776,
    -0.016865298,
    -0.011767379,
    -0.026243426,
    0.0033219527,
    -0.008215444,
    -0.02270427,
    0.0064362832,
    -0.020404456,
    0.0022950221,
    0.0040214793,
    -0.034650523,
    0.02230819,
    -0.00043680484,
    0.008036571,
    -0.025630143,
    -0.0071485867,
    -0.0072316355,
    -0.021579916,
    0.0037627502,
    -5.063283e-05,
    0.008560416,
    -0.006413924,
    0.012949227,
    -0.016788637,
    -0.00045117867,
    -0.0034337493,
    -0.017120833,
    -0.006445866,
    0.0039895372,
    -0.008381542,
    0.024480237,
    -0.00021161478,
    -0.025400162,
    -0.028875435,
    -0.040834464,
    0.018513499,
    -0.0029194853,
    0.0024164012,
    0.025374608,
    -0.0019724094,
    -0.016826969,
    -0.011032716,
    -0.16027144,
    0.02509352,
    0.026498962,
    -0.021337157,
    0.037333637,
    0.011882369,
    0.019612297,
    0.0113776885,
    0.016737532,
    -0.011933477,
    0.020736651,
    0.016533103,
    -0.02552793,
    -0.0141438525,
    0.0014773107,
    -0.021541586,
    -0.019586744,
    0.025119074,
    0.043287598,
    0.019458978,
    0.024007497,
    -0.016941959,
    0.009403681,
    -0.028773222,
    -4.556705e-05,
    0.0051618037,
    -0.008394319,
    0.0036573422,
    -0.023292,
    -0.00019903768,
    0.015843159,
    -0.005893272,
    0.012936451,
    0.0038745468,
    0.01533209,
    0.009384517,
    0.0065257205,
    0.010195839,
    -0.0050084824,
    0.012189011,
    0.015510964,
    0.016328676,
    0.011128542,
    -0.012291225,
    -0.010502481,
    0.037768047,
    0.017465806,
    -0.01838573,
    0.019727288,
    -0.019228995,
    0.019203441,
    -0.028108831,
    0.010604695,
    0.007551054,
    0.025515152,
    0.015498187,
    0.010758016,
    0.024889091,
    -0.0016154592,
    0.002256692,
    -0.011543786,
    -0.025144627,
    0.033117313,
    -0.00010570757,
    -0.01842406,
    -0.015651507,
    -0.011454348,
    0.0091800885,
    -0.012022913,
    0.001707292,
    -0.018717926,
    -0.06245271,
    0.0012569119,
    -0.036439266,
    0.017823555,
    -0.0036317888,
    -0.016303122,
    -0.009870033,
    0.0063819825,
    -0.013108936,
    -0.009627274,
    0.022576502,
    -0.005791058,
    0.014016085,
    -0.00627338,
    0.0023541145,
    -0.0031334958,
    0.020557776,
    -0.0029274707,
    -0.0067205657,
    0.027188905,
    -0.03406279,
    -0.020097815,
    -0.014335504,
    0.033526167,
    0.0066502937,
    -0.0064330893,
    0.019497307,
    -0.007154975,
    0.007864084,
    0.013415578,
    0.010004188,
    -0.021004962,
    0.01804076,
    0.038074687,
    0.02685671,
    -0.009122593,
    0.009799761,
    0.0065001673,
    0.017964099,
    -0.018219633,
    0.031737424,
    0.0052320757,
    0.026882263,
    -0.018654043,
    0.01591982,
    0.00976143,
    -0.013632783,
    0.021158284,
    0.021260498,
    0.012150681,
    0.0015819203,
    0.01533209,
    0.0051649977,
    0.018590158,
    -0.014488825,
    -0.08969273,
    -0.006829168,
    0.02568125,
    0.018845694,
    0.0029769808,
    0.037972476,
    -0.0051106964,
    0.006308516,
    0.008355989,
    0.02249984,
    -0.0016561851,
    -0.03764028,
    -0.0063532344,
    -0.009307856,
    0.011703495,
    -0.008221833,
    -0.0047305883,
    -0.015983703,
    -0.030791946,
    -2.4343186e-05,
    -0.012610643,
    -0.03360283,
    0.022359297,
    -0.00544928,
    -0.005289571,
    -0.01967618,
    -0.030025342,
    0.007295519,
    0.026422301,
    -0.010936891,
    0.011032716,
    -0.018462392,
    -0.016481996,
    -0.015753722,
    -0.008592359,
    0.006413924,
    -0.014718805,
    0.0036669248,
    0.004117305,
    -0.021516033,
    0.015740946,
    -0.0012984363,
    -0.002307799,
    -0.04908824,
    0.013377248,
    -0.009416458,
    0.0018733897,
    0.00641073,
    0.0018142973,
    -0.032325156,
    -0.016456442,
    0.012252895,
    -0.0059795147,
    -0.011882369,
    -0.0005537919,
    -0.027648868,
    0.0155620705,
    -0.02941206,
    -0.015102108,
    -0.0020937885,
    -0.022372074,
    -0.011741825,
    -0.014987118,
    -0.008828728,
    0.021349935,
    0.0017903409,
    -0.031047482,
    -0.034906056,
    -0.0020330988,
    -0.006206302,
    -0.0032676517,
    0.009243973,
    -0.0008927748,
    -0.017912991,
    -0.02626898,
    -0.011652388,
    -0.013108936,
    0.010176674,
    0.010655802,
    -0.0069761006,
    -0.009697546,
    -0.004315344,
    0.01658421,
    -0.036285944,
    0.031124141,
    0.022832036,
    0.0063180984,
    -0.0060785348,
    -0.017823555,
    -0.017708564,
    0.0009638454,
    -0.010438598,
    0.031430785,
    -0.027418887,
    0.00596035,
    0.0036860898,
    -0.0032005736,
    0.015983703,
    0.0068483334,
    -0.026575621,
    -0.022538172,
    -0.0076596565,
    -0.044667486,
    0.014693253,
    -0.02172046,
    -0.030536411,
    0.03480384,
    -0.0015795246,
    0.013952201,
    0.011543786,
    -0.002913097,
    -0.0024275808,
    -0.025234064,
    -0.008298493,
    0.0060561756,
    0.005682456,
    -0.0060242335,
    -0.007538277,
    0.018884024,
    0.0275211,
    0.0069888774,
    0.0013527374,
    0.02215487,
    0.022359297,
    0.0043185386,
    0.0129428385,
    -0.036439266,
    0.0023333523,
    0.002930665,
    0.013530569,
    -0.006238244,
    -0.03314287,
    0.026550068,
    -0.018500721,
    -0.0012026108,
    0.025438491,
    0.004414364,
    -0.030102002,
    -0.001981992,
    0.028313259,
    0.012201788,
    0.046507336,
    -0.019510085,
    -0.03493161,
    0.022014325,
    -0.023394214,
    0.012176234,
    0.01419496,
    -0.010975221,
    0.008707349,
    -0.004069392,
    0.015779275,
    0.0022279443,
    0.021043293,
    -0.019190665,
    -0.019944493,
    0.002552154,
    -0.018321848,
    0.009295079,
    0.03536602,
    0.024556896,
    0.017989652,
    0.051209178,
    0.0024084158,
    0.019599522,
    0.021081623,
    -0.014131076,
    0.016022034,
    -0.015089331,
    0.020672766,
    0.0023541145,
    0.0047114235,
    -0.017938545,
    -0.020864418,
    -0.002877961,
    0.010496093,
    0.025732357,
    0.004433529,
    -0.008956496,
    -0.005011677,
    -0.0052320757,
    0.0275211,
    -0.010125567,
    -0.010298054,
    -0.00709748,
    0.017849108,
    0.02249984,
    0.013811657,
    -0.012195399,
    0.017925767,
    0.013453908,
    0.015740946,
    0.01658421,
    0.018321848,
    0.026831156,
    0.0014413762,
    0.002170449,
    0.0031095394,
    -0.02556626,
    -0.017325262,
    0.01882014,
    0.020417232,
    0.017964099,
    0.007602161,
    -0.007876861,
    -0.02693337,
    -0.013747773,
    0.021298828,
    -0.029028757,
    -0.038764633,
    -0.020877196,
    0.01540875,
    0.0065065552,
    0.03344951,
    -0.004478248,
    0.0025633336,
    0.0009846076,
    -0.0063500404,
    0.0024946588,
    0.015958149,
    -0.012706469,
    0.023432544,
    0.008656242,
    -0.0036477596,
    0.04433529,
    -0.02120939,
    0.007582996,
    0.006528915,
    0.004532549,
    -0.0073338496,
    0.011307416,
    -0.006880275,
    0.019791173,
    0.021426596,
    -0.00441117,
    -0.025617367,
    -0.015178769,
    -0.008407095,
    0.008726514,
    8.4596e-05,
    -0.016839745,
    0.037844706,
    0.008055735,
    -0.014246066,
    0.01838573,
    5.2704057e-05,
    0.039301258,
    -0.0035551284,
    -0.009512284,
    -0.021694906,
    -0.011064658,
    0.036924783,
    -0.0027438053,
    0.022729823,
    -0.026831156,
    -0.035596002,
    -0.002243915,
    -0.018321848,
    0.019970046,
    -0.012744799,
    0.0013040261,
    0.020762205,
    0.010163898,
    0.023790292,
    -0.00015861126,
    -0.02372641,
    -0.013620006,
    0.037333637,
    0.0034944387,
    -0.021158284,
    -0.018027982,
    -0.008407095,
    0.011786544,
    -0.01933121,
    0.007397733,
    0.0020570552,
    -0.025464045,
    -0.011633223,
    -0.008554028,
    0.008036571,
    0.034011684,
    0.0047497535,
    0.008796786,
    -0.024761325,
    -0.0056888442,
    0.0076404912,
    -0.002133716,
    0.028159937,
    -0.027725529,
    -0.031251907
  ],
  "vectorWeights": null
}

### Update

In [ ]:
client.data_object.update(
    uuid=object_uuid,
    class_name="Question",
    data_object={
        'answer':"Florence, Italy"
    })

In [ ]:
data_object = client.data_object.get_by_id(
    object_uuid,
    class_name='Question',
)

json_print(data_object)

{
  "class": "Question",
  "creationTimeUnix": 1708072945269,
  "id": "d470073f-f781-43ac-8e4d-ea68bcb3dc76",
  "lastUpdateTimeUnix": 1708073140282,
  "properties": {
    "answer": "Florence, Italy",
    "category": "Culture",
    "question": "Leonardo da Vinci was born in this country."
  },
  "vectorWeights": null
}

### Delete

In [ ]:
json_print(client.query.aggregate("Question").with_meta_count().do())

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 11
          }
        }
      ]
    }
  }
}

In [ ]:
client.data_object.delete(uuid=object_uuid, class_name="Question")

In [ ]:
json_print(client.query.aggregate("Question").with_meta_count().do())

{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 10
          }
        }
      ]
    }
  }
}